### 교차 검증

-   부족한 데이터셋 및 특정 데이터에 과대적합되는 문제 해결하기 위한 방안
-   학습 데이터셋을 일정 크기의 데이터로 n개 분리 후 1/n은 검증용, 나머지는 학습용으로 사용


[1] 모듈 로딩 및 데이터 준비 <hr>


In [110]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])


In [111]:
# KFold 인스턴스 생성 => 데이터를 2개로 분할해주는 객체
k_fold = KFold(n_splits=2)


In [112]:
# 데이터 분할
datasets = k_fold.split(X, y)

for dataset in datasets:
    print(dataset)


(array([2, 3]), array([0, 1]))
(array([0, 1]), array([2, 3]))


In [113]:
### perch.csv 파일 데이터 기본 5등분
import pandas as pd

perchDF = pd.read_csv("../data/perch3.csv")
perchDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Weight  56 non-null     float64
 1   Length  56 non-null     float64
 2   Height  56 non-null     float64
 3   Width   56 non-null     float64
dtypes: float64(4)
memory usage: 1.9 KB


In [114]:
# perchDF => 5등분
fold_5 = KFold(n_splits=3)

datasets = fold_5.split(perchDF)

for index, (train, test) in enumerate(datasets):
    print(f"{index}, {train.shape} {test.shape}")


0, (37,) (19,)
1, (37,) (19,)
2, (38,) (18,)


In [115]:
# perchDF => 5등분
fold_5 = KFold(n_splits=5)

datasets = fold_5.split(perchDF)

for index, (train, test) in enumerate(datasets):
    print(f"{index}, {train.shape} {test.shape}")


0, (44,) (12,)
1, (45,) (11,)
2, (45,) (11,)
3, (45,) (11,)
4, (45,) (11,)


In [116]:
## 타겟이 분류인 경우
irisDF = pd.read_csv("../data/iris.csv")
irisDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [117]:
k_fold = KFold(n_splits=3, shuffle=True)
ret = k_fold.split(irisDF[irisDF.columns[:-1]])


In [118]:
from sklearn.linear_model import LogisticRegression


In [119]:
train_score3 = []
for idx, (train, test) in enumerate(ret):
    # 학습용, 테스트용 인덱스 변환
    train_idx = train.tolist()
    test_idx = test.tolist()

    # 인덱스에 해당하는 데이터셋 추출
    trainDF = irisDF.iloc[train_idx]
    testDF = irisDF.iloc[test_idx]
    # print(trainDF['species'].value_counts()/trainDF.shape[0])
    # print(testDF['species'].value_counts()/testDF.shape[0])

    x_train = trainDF[trainDF.columns[:-1]]
    y_train = trainDF[trainDF.columns[-1]]

    x_test = testDF[testDF.columns[:-1]]
    y_test = testDF[testDF.columns[-1]]

    # 분류 모델 학습
    log_model = LogisticRegression(max_iter=1000, solver="liblinear")
    log_model.fit(x_train, y_train)

    # 훈련 및 검증용 성능
    train_score = log_model.score(x_train, y_train)
    test_score = log_model.score(x_test, y_test)

    train_score3.append(test_score)


In [125]:
sum(train_score3) / 3


0.9466666666666667